In [2]:
import requests
import pandas as pd

# Enter data here:
isbn = "9783423218368"
genre = "Thriller"
date = "2024-03-20"
rating = 4.0     # x out of 5 stars


def get_book_info(isbn):
    url = f"https://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&format=json&jscmd=data"
    response = requests.get(url)
    data = response.json()

    # extract book info
    book_info = data.get(f'ISBN:{isbn}', {})
    title = book_info.get('title', 'N/A')
    authors = ", ".join(author['name'] for author in book_info.get('authors', []))
    number_of_pages = book_info.get('number_of_pages', 'N/A')
    cover_url = book_info.get('cover', {}).get('large', 'N/A')

    return title, authors, number_of_pages, cover_url

# call book info
book_info = get_book_info(isbn)

# make dataframe
data = {
    "title": [book_info[0]],
    "author": [book_info[1]],
    "number_of_pages": [book_info[2]],
    "genre": genre,
    "url": [book_info[3]],
    "finishing_date": pd.to_datetime(date),
    "rating": rating
}

df = pd.DataFrame(data)


# Check title available?
if df.at[0, 'title'] == 'N/A':
    value = input("Enter 'title' here: ")
    df.at[0, 'title'] = value

# Check author available?
if df.at[0, 'author'] == 'N/A':
    value = input("Enter 'author' here: ")
    df.at[0, 'author'] = value

# Check number of pages available?
if df.at[0, 'number_of_pages'] == 'N/A':
    value = input("Enter 'number_of_pages' here: ")
    df.at[0, 'number_of_pages'] = value

# Check book cover available?
if df.at[0, 'url'] == 'N/A':
    value = input("Enter 'url' here: ")    # takes local path
    df.at[0, 'url'] = value

    
df

,title,author,number_of_pages,genre,url,finishing_date,rating
0,Liebes Kind,Romy Hausmann,427,Thriller,https://covers.openlibrary.org/b/id/14571593-L...,2024-03-20,4.0


In [2]:
from sqlalchemy import create_engine
import psycopg2

# connect to database
dbname = 'dbname'
user = 'username'
password = 'mypassword'
host = 'host'  
port = 'port'  

engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')


# load dataframe to staging table
try:
    df.to_sql('books_staging', con=engine, if_exists='append', index=False)
    print("Data successfully loaded to staging table.")
except Exception as e:
    print(f"Error: {e}")

engine.dispose()


Data successfully loaded to staging table.
